In [14]:
import pandas as pd
import numpy as np

In [25]:
data = pd.read_csv('3-B_EXAM4.csv', encoding='GB2312')
data

,日期,市场指数,组合1,组合2,组合3,组合4,组合5,无风险利率,市场溢酬因子,市值因子,账面市值比因子
0,20090109,1904.86,1727.01,1281.30,1591.43,2469.13,1822.58,0.000043,0.0204,0.0083,0.0010
1,20090112,1900.35,1715.86,1300.27,1590.85,2488.00,1829.90,0.000042,0.0044,0.0102,-0.0021
2,20090113,1863.37,1667.97,1273.80,1551.21,2407.01,1774.56,0.000041,-0.0249,-0.0019,0.0074
3,20090114,1928.87,1750.08,1329.63,1607.21,2463.77,1817.24,0.000041,0.0427,0.0029,-0.0022
4,20090115,1920.21,1739.64,1334.24,1614.49,2456.65,1814.10,0.000040,0.0033,0.0079,-0.0031
...,...,...,...,...,...,...,...,...,...,...,...
2422,20181224,2527.01,1289.12,1634.66,1869.08,7276.91,5275.89,0.000088,0.0056,-0.0008,-0.0045
2423,20181225,2504.82,1259.94,1605.20,1846.48,7256.91,5251.07,0.000090,-0.0088,-0.0034,-0.0057
2424,20181226,2498.29,1250.42,1598.94,1843.40,7216.06,5193.21,0.000091,-0.0030,0.0048,0.0030
2425,20181227,2483.09,1236.36,1587.71,1831.41,7233.59,5144.72,0.000091,-0.0080,-0.0095,0.0024


In [16]:
p1 = data.iloc[:, 2].values
p2 = data.iloc[:, 3].values
p3 = data.iloc[:, 4].values
p4 = data.iloc[:, 5].values
p5 = data.iloc[:, 6].values
rf = data.iloc[:, 7].values
mkt = data.iloc[:, 8].values
p1

array([1727.01, 1715.86, 1667.97, ..., 1250.42, 1236.36, 1225.71])

In [17]:
r1 = np.log(p1[1:]) - np.log(p1[:-1])
r2 = np.log(p2[1:]) - np.log(p2[:-1])
r3 = np.log(p3[1:]) - np.log(p3[:-1])
r4 = np.log(p4[1:]) - np.log(p4[:-1])
r5 = np.log(p5[1:]) - np.log(p5[:-1])
r1

array([-0.00647718, -0.02830709,  0.04805418, ..., -0.00758461,
       -0.01130792, -0.00865131])

In [18]:
rexc1 = r1 - rf[1:]
rexc2 = r2 - rf[1:]
rexc3 = r3 - rf[1:]
rexc4 = r4 - rf[1:]
rexc5 = r5 - rf[1:]
rexc1

array([-0.00651918, -0.02834809,  0.04801318, ..., -0.00767561,
       -0.01139892, -0.00874331])

In [27]:
R = np.concatenate([rexc1[:, None], 
                    rexc2[:, None],
                    rexc3[:, None], 
                    rexc4[:, None],
                    rexc5[:, None]], 
                   axis=1)
type(R)

numpy.ndarray

In [28]:
Cov_Sample = np.mat(np.cov(R, rowvar=False))
Cov_Sample  ##方法1：样本方差-协方差矩阵
type(np.cov(R, rowvar=False))

numpy.ndarray

In [21]:
X = np.mat(np.concatenate([np.ones((len(mkt)-1, 1)), mkt[1:, None]], axis=1))
X  #这里就是的X就是常数项和市场超额收益率的系数

matrix([[ 1.    ,  0.0044],
        [ 1.    , -0.0249],
        [ 1.    ,  0.0427],
        ...,
        [ 1.    , -0.003 ],
        [ 1.    , -0.008 ],
        [ 1.    ,  0.0039]])

In [29]:
Y = np.mat(R)
AB_hat = (X.T*X).I*(X.T*Y)
ALPHA = AB_hat[0]#这里取的是AB_hat的第一行，总之就是常数项的系数
BETA = AB_hat[1]#这里取的是AB_hat的第二行，总之就是市场超额收益率的系数
RESD = Y - X*AB_hat
covfactor = np.cov(mkt[1:])
covresidual = np.diag(np.diag(np.cov(RESD, rowvar=False)))
Cov_Factor = BETA.T*covfactor*BETA + covresidual
Cov_Factor##方法3：因子模型估计法
covresidual

array([[9.02498281e-05, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00],
       [0.00000000e+00, 6.25901678e-05, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 4.46072446e-05, 0.00000000e+00,
        0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 7.58804669e-05,
        0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        1.13345790e-04]])

In [23]:
c = 0.5
Cov_Shrink = c*Cov_Sample + (1-c)*Cov_Factor
##这里要注意c是加反的
Cov_Shrink

matrix([[0.00036161, 0.00030257, 0.00026619, 0.00022084, 0.00020533],
        [0.00030257, 0.00036965, 0.00028306, 0.00024205, 0.00023079],
        [0.00026619, 0.00028306, 0.00029853, 0.0002197 , 0.00020996],
        [0.00022084, 0.00024205, 0.0002197 , 0.00026741, 0.00020887],
        [0.00020533, 0.00023079, 0.00020996, 0.00020887, 0.0002925 ]])

In [37]:
uhat = np.mean(R, axis=0)
A = np.mat(np.concatenate([uhat[:, None], np.ones((len(uhat), 1))], axis=1)).T
up = np.mean(uhat)
b = np.mat(np.array([up, 1])[:, None])

omega_Shrink = Cov_Shrink.I*A.T*(A*Cov_Shrink.I*A.T).I*b
print(omega_Shrink)
omega_Sample = Cov_Sample.I*A.T*(A*Cov_Sample.I*A.T).I*b
print()
print(omega_Sample)
b

[[ 0.32532288]
 [-0.21248216]
 [ 0.38898507]
 [ 0.23881412]
 [ 0.25936009]]

[[ 0.39904587]
 [-0.32861957]
 [ 0.38202395]
 [ 0.23869463]
 [ 0.30885513]]


matrix([[7.49300704e-05],
        [1.00000000e+00]])